In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import GetOldTweets3 as got
import tweepy

import datetime
import json

%matplotlib inline

In [2]:
def get_tweets(username, since_date='2018-01-01', until_date='now'):
    
    '''Get all tweets from a user in a given time interval.
    
    Args:
        username (str): Username
        since_date (str): The start date of the interval
        until_date (str): The end date
    
    Returns:
        tweets: List of GetOldTweets3.models.Tweet.Tweet objects
    '''

    if until_date == 'now':
        until_date = datetime.datetime.now().strftime('%Y-%m-%d')

    # Creation of query object
    tweetCriteria = got.manager.TweetCriteria().setUsername(username)\
        .setSince(since_date).setUntil(until_date).setTopTweets(False)

    # Creation of list that contains all tweets
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)

    return tweets

In [3]:
tweets = get_tweets('strawlab')

In [5]:
def tweet_to_dict(tweet):
    
    '''Transforms a GetOldTweets3-Tweet object to dictionary
    
    Args:
        tweet: GetOldTweets3.models.Tweet.Tweet object
        
    Returns:
        tweet_dict: Dictionary representation of the Tweet object
    '''

    tweet_dict = {
        'id': tweet.id,
        'permalink': tweet.permalink,
        'username': tweet.username,
        'to': tweet.to,
        'text': tweet.text,
        'date': tweet.date.strftime('%Y-%m-%d-%H-%M-%S'),
        'retweets': tweet.retweets,
        'favorites': tweet.favorites,
        'mentions': tweet.mentions,
        'hashtags': tweet.hashtags,
        'geo': tweet.geo
    }
    
    return tweet_dict

In [8]:
tweets_dict = list(map(tweet_to_dict, tweets))

In [ ]:
def get_API():
    
    '''Gets the Twitter API and authenticates with the App "getMdBList".
    
    Returns:
        api: Twitter API
    '''
    
    # read acess key from external file
    with open("app_data", "r") as f:
        consumer_key = f.readline().splitlines()[0]
        consumer_secret = f.readline().splitlines()[0]

    auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)

    api = tweepy.API(auth)
    
    return api


def retrieve_members_bundestag(api):
    
    '''Retrieves the members of the Twitter list "MdB (Bundestag)" by "wahl_beobachter"
    https://twitter.com/i/lists/912241909002833921
    
    Returns:
        members: List of members
    '''
    
    return list(tweepy.Cursor(api.list_members, list_id=912241909002833921).items())


# initialize Twitter API
api = get_API()

# get list of Bundestag members from https://twitter.com/i/lists/912241909002833921
members_bundestag = retrieve_members_bundestag(api)

# convert tweepy-User objects to dicts
members_bundestag = list(map(lambda user: user._json, members_bundestag))